# Top and Bottom Performing
Let's look at how we might get the top performing stocks for a single period. For this example, we'll look at just a single month of closing prices:

In [39]:
%%writefile project_tests.py

from collections import OrderedDict
import pandas as pd
from tests1 import project_test, generate_random_tickers, generate_random_dates, assert_output


@project_test
def test_date_top_industries(fn):
    tickers = generate_random_tickers(10)
    dates = generate_random_dates(2)

    fn_inputs = {
        'prices': pd.DataFrame(
            [
                [21.050810483942833, 17.013843810658827, 10.984503755486879, 11.248093428369392, 12.961712733997235,
                 482.34539247360806, 35.202580592515041, 3516.5416782257166, 66.405314327318209, 13.503960481087077],
                [15.63570258751384, 14.69054309070934, 11.353027688995159, 475.74195118202061, 11.959640427803022,
                 10.918933017418304, 17.9086438675435, 24.801265417692324, 12.488954191854916, 15.63570258751384]],
            dates, tickers),
        'sector': pd.Series(
            ['ENERGY', 'MATERIALS', 'ENERGY', 'ENERGY', 'TELECOM', 'FINANCIALS',
             'TECHNOLOGY', 'HEALTH', 'MATERIALS', 'REAL ESTATE'],
            tickers),
        'date': dates[-1],
        'top_n': 4}
    fn_correct_outputs = OrderedDict([
        (
            'top_industries',
            set(['ENERGY', 'HEALTH', 'TECHNOLOGY']))])

    assert_output(fn, fn_inputs, fn_correct_outputs)



Overwriting project_tests.py


In [40]:
import pandas as pd

month = pd.to_datetime('02/01/2018')
close_month = pd.DataFrame(
    {
        'A': 1,
        'B': 12,
        'C': 35,
        'D': 3,
        'E': 79,
        'F': 2,
        'G': 15,
        'H': 59},
    [month])

close_month

,A,B,C,D,E,F,G,H
2018-02-01,1,12,35,3,79,2,15,59


`close_month` gives use the prices for the month of February, 2018 for all the stocks in this universe (A, B, C, ...). Looking at these prices, we can see that the top 2 performing stocks for that month was E and H with the prices 79 and 59.

To get this using code, we can use the [`Series.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.nlargest.html) function. This function returns the items with the *n* largest numbers. For the example we just talked about, our *n* is 2.

In [41]:
try:
    # Attempt to run nlargest
    close_month.nlargest(2)
except TypeError as err:
    print('Error: {}'.format(err))

Error: nlargest() missing 1 required positional argument: 'columns'


What happeened here? It turns out we're not calling the [`Series.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.nlargest.html) function, we're actually calling [`DataFrame.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.nlargest.html), since `close_month` is a DataFrame. Let's get the Series from the dataframe using `.loc[month]`, where `month` is the 2018-02-01 index created above.

In [42]:
close_month.loc[month].nlargest(2)

E    79
H    59
Name: 2018-02-01 00:00:00, dtype: int64

Perfect! That gives us the top performing tickers for that month. Now, how do we get the bottom performing tickers? There's two ways to do this. You can use Panda's [`Series.nsmallest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.Series.nsmallest.html) function or just flip the sign on the prices and then apply [`DataFrame.nlargest`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.nlargest.html). Either way is fine. For this course, we'll flip the sign with nlargest. This allows us to reuse any funtion created with nlargest to get the smallest.

To get the bottom 2 performing tickers from `close_month`, we'll flip the sign.

In [43]:
(-1 * close_month).loc[month].nlargest(2)

A   -1
F   -2
Name: 2018-02-01 00:00:00, dtype: int64

That gives us the bottom performing tickers, but not the actual prices. To get this, we can flip the sign from the output of nlargest.

In [44]:
(-1 * close_month).loc[month].nlargest(2) *-1

A    1
F    2
Name: 2018-02-01 00:00:00, dtype: int64

Now you've seen how to get the top and bottom performing prices in a single month. Let's see if you can apply this knowledge.
## Quiz
Implement `date_top_industries` to find the top performing closing prices and return their sectors for a single date. The function should only return the [set](https://docs.python.org/3/tutorial/datastructures.html#sets) of sectors, there shouldn't be any duplicates returned.

- The number of top performing prices to look at is represented by the parameter `top_n`.
- The `date` parameter is the date to look for the top performing prices in the `prices` DataFrame.
- The sector information for each ticker is located in the `sector` parameter.

For example:
```
                 Prices
               A         B         C         D         E
2013-07-08     2         2         7         2         6
2013-07-09     5         3         6         7         5
...            ...       ...       ...

           Sector
A       "Utilities"       
B       "Health Care"       
C       "Real Estate"
D       "Real Estate"
E       "Information Technology"

Date:  2013-07-09
Top N: 3
```
The set created from the function `date_top_industries` should be the following:
```
{"Utilities", "Real Estate"}
```
*Note: Stock A and E have the same price for the date, but only A's sector got returned. We'll keep it simple and only take the first occurrences of ties.*

In [49]:
import project_tests
from collections import OrderedDict
import pandas as pd
from tests1 import project_test, generate_random_tickers, generate_random_dates, assert_output


def date_top_industries(prices, sector, date, top_n):
    """
    Get the set of the top industries for the date
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    sector : Series
        Sector name for each ticker
    date : Date
        Date to get the top performers
    top_n : int
        Number of top performers to get
    
    Returns
    -------
    top_industries : set
        Top industries for the date
    """
    # TODO: Implement Function
    print(prices)
    #print(sector)
    #print(date)
    
    
    top_n_stock=prices.loc[date].nlargest(top_n).index
    top_sectors=set(sector.loc[top_n_stock])
    print(" Top n stock - ",   top_n_stock)
    
    return top_sectors


project_tests.test_date_top_industries(date_top_industries)

                  IRMU         GLB        JBYW          ZCS         JNH  \
2007-09-17 21.05081048 17.01384381 10.98450376  11.24809343 12.96171273   
2007-09-18 15.63570259 14.69054309 11.35302769 475.74195118 11.95964043   

                    YFZ        FZDY           XEU        IHWC         LEP  
2007-09-17 482.34539247 35.20258059 3516.54167823 66.40531433 13.50396048  
2007-09-18  10.91893302 17.90864387   24.80126542 12.48895419 15.63570259  
 Top n stock -  Index(['ZCS', 'XEU', 'FZDY', 'IRMU'], dtype='object')
Tests Passed


## Quiz Solution
If you're having trouble, you can check out the quiz solution [here](top_and_bottom_performing_solution.ipynb).